In [ ]:
!pip install bert-score
!pip install datasets
!pip install sentence-transformers datasets accelerate transformers wandb
!pip install sentence-transformers torch sklearn bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install peft -q

Baseline BAAI/bge-m3 model

In [ ]:
# @title
# ============================================================
# CLEAN QA → Fine-tune + Evaluation
# VERSION 5 — ТАЗА ДАТАСЕТ (сегменттелмеген)
#
# ӨЗГЕРІСТЕР (v4 → v5):
#   - DATA_PATH  → baseline_15000.json  (таза, @@-сіз)
#   - seg_train  жойылды → тек cln_train қолданылады
#   - clean_segmented_text() тек қауіпсіздік үшін сақталды
#     (таза деректе @@ жоқ болса да зиян жоқ)
#   - train_examples: cln_q + cln_a  (таза мәтін)
#   - Барлық басқа логика v4-пен БІРДЕЙ
#
# ӨЗГЕРІСТЕР (v5 → v5-bge):
#   - MODEL_NAME  → BAAI/bge-m3
#   - OUTPUT_PATH → fine-tuned-kz-model-bge-m3
#   - encode_query: BGE-M3 prefix қажет етпейді → prefix алынды
#   - encode_passage: өзгеріссіз (prefix жоқ болатын)
#   - train_examples: e5_wrap_query() жойылды → cln_q тікелей
#   - detect_lora_targets(): авто-анықтау функциясы қосылды
#
# Metrics:
#   Exact@1 | TokenF1@1 | MeanCos@1(QSim)
#   Semantic@1(ans_cos>=0.85) | BERTScoreF1@1 (optional)
#
# Requires: pip install peft
# ============================================================

import os
os.environ["WANDB_DISABLED"]              = "true"
os.environ["TOKENIZERS_PARALLELISM"]      = "false"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"]     = "expandable_segments:True"

import pandas as pd
import numpy as np
import time, re, json, glob, random
import torch
from pathlib import Path
from collections import Counter

# ── PEFT / LoRA ──────────────────────────────────────────────
try:
    from peft import LoraConfig, get_peft_model, TaskType
    HAS_PEFT = True
except ImportError:
    HAS_PEFT = False
    print("⚠️  peft not installed. Run:  pip install peft")
    print("    Falling back to frozen-backbone training.")

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# BERTScore — optional
try:
    from bert_score import score as bert_score_fn
    HAS_BERT_SCORE = True
except ImportError:
    bert_score_fn  = None
    HAS_BERT_SCORE = False


# ================================================================
# CONFIG
# ================================================================
DATA_PATH   = "baseline_15000.json"

# ✅ ӨЗГЕРІС 1: Модель ауыстырылды
MODEL_NAME  = "BAAI/bge-m3"                  # бұрын: intfloat/multilingual-e5-large-instruct
OUTPUT_PATH = "fine-tuned-kz-model-bge-m3"   # бұрын: fine-tuned-kz-model-e5-large

TEST_SIZE    = 0.4
EPOCHS       = 3
WARMUP_STEPS = 100
BATCH_SIZE   = 16
SEM_THR      = 0.85
SEED         = 42
USE_FP16     = torch.cuda.is_available()

LORA_R       = 16
LORA_ALPHA   = 32
LORA_DROPOUT = 0.05
LORA_TARGETS = ["query", "value"]


# ================================================================
# REPRODUCIBILITY
# ================================================================
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)


# ================================================================
# FILE UTILITIES
# ================================================================
def find_data_path(p: str) -> str:
    if Path(p).exists():
        return p
    for c in [f"/content/{p}", f"/content/drive/MyDrive/{p}"]:
        if Path(c).exists():
            return c
    name = Path(p).name
    hits = glob.glob(f"**/{name}", recursive=True)
    if hits:
        return hits[0]
    near = glob.glob("**/*.json", recursive=True)
    raise FileNotFoundError(
        f"File not found: {p}\nPWD: {Path.cwd()}\n"
        f"Nearby .json files:\n" + "\n".join(near[:30])
    )


def _normalize_records(data: list) -> list:
    out = []
    for x in data:
        if not isinstance(x, dict):
            continue
        q = x.get("question") or x.get("instruction") or ""
        a = x.get("answer")   or x.get("response")    or ""
        q, a = str(q).strip(), str(a).strip()
        if q and a:
            out.append({"question": q, "answer": a})
    if not out:
        raise ValueError("No valid question/answer pairs found.")
    return out


def load_qa_records(path: str) -> list:
    text = Path(path).read_text(encoding="utf-8", errors="ignore").strip()
    if not text:
        raise ValueError(f"File is empty: {path}")

    if text[0] == "[":
        try:
            return _normalize_records(json.loads(text))
        except Exception:
            pass

    lines = [ln.strip().rstrip(",") for ln in text.splitlines() if ln.strip()]
    if lines and lines[0].startswith("{"):
        recs, ok = [], True
        for ln in lines:
            try:
                recs.append(json.loads(ln))
            except Exception:
                ok = False; break
        if ok and recs:
            return _normalize_records(recs)

    objs, buf, depth = [], [], 0
    in_str = esc = started = False
    for ch in text:
        if not started:
            if ch == "{":
                started, depth, buf = True, 1, ["{"]
            continue
        buf.append(ch)
        if in_str:
            if esc:          esc = False
            elif ch == "\\": esc = True
            elif ch == '"':  in_str = False
        else:
            if   ch == '"': in_str = True
            elif ch == "{": depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    try:
                        objs.append(json.loads("".join(buf)))
                    except Exception:
                        pass
                    buf, started = [], False

    if not objs:
        raise ValueError(f"Could not parse JSON. Preview:\n{text[:400]}")
    return _normalize_records(objs)


# ================================================================
# TEXT UTILITIES
# ================================================================
def clean_segmented_text(text: str) -> str:
    # ✅ Таза деректе @@ болмайды, бірақ қауіпсіздік үшін сақтаймыз
    t = re.sub(r"@@\s*", "", str(text))
    t = re.sub(r"\s+", " ", t).strip()
    t = re.sub(r"\s+([,.!?;:])", r"\1", t)
    t = re.sub(r"\s*([-/])\s*", r"\1", t)
    return t


def normalize_text(t: str) -> str:
    return re.sub(r"\s+", " ", str(t).lower().strip())


KZ_PAT = re.compile(
    r"[a-zA-Zа-яА-ЯәғқңөұүһіӘҒҚҢӨҰҮҺІ0-9]+"
)

def tokens(text: str) -> list:
    return KZ_PAT.findall(normalize_text(text))


def token_f1(pred: str, gold: str) -> float:
    p, g = tokens(pred), tokens(gold)
    if not p and not g: return 1.0
    if not p or  not g: return 0.0
    pc, gc = Counter(p), Counter(g)
    inter  = sum((pc & gc).values())
    if inter == 0: return 0.0
    prec = inter / len(p)
    rec  = inter / len(g)
    return (2 * prec * rec) / (prec + rec + 1e-12)


# ================================================================
# BGE-M3 ENCODING
#
# ✅ ӨЗГЕРІС 2: BGE-M3 — prefix ҚАЖЕТ ЕМЕС (екі жақта да)
#
#   E5-instruct (бұрынғы):
#     query   → "Instruct: ...\nQuery: {text}"  (міндетті prefix)
#     passage → тікелей мәтін
#
#   BGE-M3 (қазіргі):
#     query   → тікелей мәтін  (prefix жоқ)
#     passage → тікелей мәтін  (prefix жоқ)
#
#   BAAI ресми нұсқаулығы:
#     "No instruction needed for either queries or passages."
# ================================================================

def _nan_guard(vecs: np.ndarray, label: str) -> np.ndarray:
    bad = ~np.isfinite(vecs).all(axis=1)
    if bad.any():
        print(f"  ⚠ {bad.sum()} {label} vectors had NaN/Inf → zeroed.")
        vecs[bad] = 0.0
    return vecs

def encode_query(mdl: SentenceTransformer, texts: list) -> np.ndarray:
    # ✅ BGE-M3: e5_wrap_query() жоқ — мәтін тікелей
    vecs = mdl.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=32,
        show_progress_bar=False,
    )
    return _nan_guard(vecs, "query")

def encode_passage(mdl: SentenceTransformer, texts: list) -> np.ndarray:
    vecs = mdl.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=32,
        show_progress_bar=False,
    )
    return _nan_guard(vecs, "passage")


# ================================================================
# LOAD DATA
# ================================================================
data_path = find_data_path(DATA_PATH)
rows      = load_qa_records(data_path)

clean_data = pd.DataFrame(rows)
clean_data["question"] = clean_data["question"].apply(clean_segmented_text)
clean_data["answer"]   = clean_data["answer"].apply(clean_segmented_text)

print(f"[LOADED] rows={len(clean_data)} | path={data_path}")
print(f"[DATA]   Үлгі сұрақ : {clean_data['question'].iloc[0][:80]}")
print(f"[DATA]   Үлгі жауап : {clean_data['answer'].iloc[0][:80]}")


# ================================================================
# TRAIN / TEST SPLIT
# ================================================================
idx = np.arange(len(clean_data))
train_idx, test_idx = train_test_split(
    idx, test_size=TEST_SIZE, random_state=SEED, shuffle=True
)

cln_train = clean_data.iloc[train_idx].reset_index(drop=True)
cln_test  = clean_data.iloc[test_idx].reset_index(drop=True)

print(f"[SPLIT] train={len(cln_train)} | test={len(cln_test)}")


# ================================================================
# LOAD BASE MODEL
# ================================================================
print(f"\n[MODEL] Loading {MODEL_NAME} ...")
model = SentenceTransformer(MODEL_NAME)


# ================================================================
# LoRA TARGET MODULES АВТОМАТТЫ АНЫҚТАУ
#
# ✅ ӨЗГЕРІС 3: BGE-M3 нұсқасына байланысты attention layer
#    атаулары "query"/"value" немесе "q_proj"/"v_proj" болуы мүмкін.
#    Функция backbone графын тексеріп дұрыс атауды өзі табады.
# ================================================================
def detect_lora_targets(st_model: SentenceTransformer) -> list:
    backbone     = st_model[0].auto_model
    linear_names = set()
    for name, module in backbone.named_modules():
        if isinstance(module, torch.nn.Linear):
            linear_names.add(name.split(".")[-1])

    for pair in [["query", "value"], ["q_proj", "v_proj"], ["q", "v"]]:
        if all(t in linear_names for t in pair):
            print(f"[LoRA] Detected target modules: {pair}")
            return pair

    fallback = list(linear_names)[:4]
    print(f"[LoRA] Standard targets not found. Fallback: {fallback}")
    return fallback


# ================================================================
# APPLY LoRA TO THE BACKBONE
# ================================================================
def apply_lora(st_model: SentenceTransformer) -> SentenceTransformer:
    backbone = st_model[0].auto_model

    # ✅ ӨЗГЕРІС 4: Hardcode LORA_TARGETS орнына авто-анықтау
    detected_targets = detect_lora_targets(st_model)

    lora_cfg = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=detected_targets,
        bias="none",
    )

    peft_backbone = get_peft_model(backbone, lora_cfg)

    total  = sum(p.numel() for p in peft_backbone.parameters())
    train_ = sum(p.numel() for p in peft_backbone.parameters() if p.requires_grad)
    print(f"[LoRA] Total params : {total/1e6:.1f}M")
    print(f"[LoRA] Trainable    : {train_/1e6:.2f}M  ({100*train_/total:.2f}% of total)")

    st_model[0].auto_model = peft_backbone
    return st_model


def freeze_backbone(st_model: SentenceTransformer) -> SentenceTransformer:
    for param in st_model[0].auto_model.parameters():
        param.requires_grad = False
    total  = sum(p.numel() for p in st_model.parameters())
    train_ = sum(p.numel() for p in st_model.parameters() if p.requires_grad)
    print(f"[FROZEN] Trainable: {train_/1e6:.2f}M / {total/1e6:.1f}M total")
    return st_model


if HAS_PEFT:
    model = apply_lora(model)
    print("[LoRA] Adapters applied. Backbone frozen.")
else:
    model = freeze_backbone(model)
    print("[FROZEN] Backbone frozen (install peft for better results).")

if torch.cuda.is_available():
    alloc      = torch.cuda.memory_allocated() / 1e9
    total_vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"[GPU] After model prep: {alloc:.2f} GB / {total_vram:.1f} GB")


# ================================================================
# BUILD TRAINING EXAMPLES
# ✅ ӨЗГЕРІС 5: e5_wrap_query() жойылды → cln_q тікелей беріледі
# ================================================================
print("\n[TRAIN] Building training examples ...")
train_examples   = []
n_train          = len(cln_train)
all_cln_answers  = cln_train["answer"].tolist()

for i in range(n_train):
    cln_q = str(cln_train.loc[i, "question"]).strip()
    cln_a = str(cln_train.loc[i, "answer"]).strip()
    if not cln_q or not cln_a:
        continue
    neg_idx = random.choice([k for k in range(n_train) if k != i])
    neg_a   = str(all_cln_answers[neg_idx]).strip()
    # ✅ BGE-M3: prefix жоқ — cln_q тікелей
    train_examples.append(
        InputExample(texts=[cln_q, cln_a, neg_a])
    )

if len(train_examples) < 2:
    raise ValueError("Not enough training pairs.")

print(f"[TRAIN] {len(train_examples)} examples | batch_size={BATCH_SIZE}")

train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
)

try:
    train_loss = losses.CachedMultipleNegativesRankingLoss(
        model, mini_batch_size=8
    )
    print("[LOSS] CachedMultipleNegativesRankingLoss")
except AttributeError:
    train_loss = losses.MultipleNegativesRankingLoss(model)
    print("[LOSS] MultipleNegativesRankingLoss")


# ================================================================
# CLEAR VRAM BEFORE TRAINING
# ================================================================
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    free = (torch.cuda.get_device_properties(0).total_memory
            - torch.cuda.memory_allocated()) / 1e9
    print(f"\n[GPU] Free VRAM before fit: {free:.2f} GB")


# ================================================================
# FINE-TUNING
# ================================================================
print(f"\n[FINE-TUNE] Epochs={EPOCHS} | warmup={WARMUP_STEPS} | fp16={USE_FP16}")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=WARMUP_STEPS,
    show_progress_bar=True,
    use_amp=USE_FP16,
    optimizer_params={"lr": 2e-4},
)


# ================================================================
# MERGE LoRA WEIGHTS & SAVE
# ================================================================
if HAS_PEFT:
    print("\n[LoRA] Merging adapter weights into base model ...")
    try:
        model[0].auto_model = model[0].auto_model.merge_and_unload()
        print("[LoRA] Merge complete.")
    except Exception as e:
        print(f"[LoRA] Merge failed ({e}) — saving with adapter weights.")

model.save(OUTPUT_PATH)
print(f"[SAVED] {OUTPUT_PATH}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()

model = SentenceTransformer(OUTPUT_PATH)
print("[LOADED] Model reloaded from disk.")


# ================================================================
# BUILD KB INDEX
# ================================================================
train_questions = cln_train["question"].tolist()
train_answers   = cln_train["answer"].tolist()

print(f"\n[KB] Indexing {len(train_questions)} training questions ...")
train_q_embeds = encode_query(model, train_questions)
print(f"[KB] Index shape: {train_q_embeds.shape}")


# ================================================================
# RETRIEVAL
# ================================================================
def ask_question(question: str, threshold: float = 0.0):
    q_clean    = clean_segmented_text(question)
    q_vec      = encode_query(model, [q_clean])
    sims       = q_vec @ train_q_embeds.T
    best_idx   = int(np.argmax(sims[0]))
    best_score = float(sims[0][best_idx])

    if not np.isfinite(best_score):
        return "Кешіріңіз, нақты жауап табылмады.", -1, 0.0
    if best_score < threshold:
        return "Кешіріңіз, нақты жауап табылмады.", -1, best_score
    return train_answers[best_idx], best_idx, best_score


# ================================================================
# EVALUATION
# ================================================================
def evaluate_system() -> dict:
    print("\n[EVAL] Running ...")
    t0 = time.time()

    gold_qs = cln_test["question"].tolist()
    gold_as = cln_test["answer"].tolist()
    n_test  = len(gold_qs)

    pred_as, qsims = [], []
    for q in gold_qs:
        pred_a, _, qsim = ask_question(q, threshold=0.0)
        pred_as.append(pred_a)
        qsims.append(qsim)

    print("[EVAL] Batch-encoding for semantic similarity ...")
    pred_vecs   = encode_passage(model, pred_as)
    gold_vecs   = encode_passage(model, gold_as)
    ans_cos_all = np.sum(pred_vecs * gold_vecs, axis=1)

    exacts, tf1s, qcos1s, semhits = [], [], [], []
    for i in range(n_test):
        exacts.append(1.0 if normalize_text(pred_as[i]) == normalize_text(gold_as[i]) else 0.0)
        tf1s.append(token_f1(pred_as[i], gold_as[i]))
        qcos1s.append(float(qsims[i]) if np.isfinite(qsims[i]) else 0.0)
        semhits.append(1.0 if float(ans_cos_all[i]) >= SEM_THR else 0.0)

    results = {
        "Exact@1":                   float(np.mean(exacts)),
        "TokenF1@1":                 float(np.mean(tf1s)),
        "MeanCos@1(QSim)":           float(np.mean(qcos1s)),
        "Semantic@1(ans_cos>=0.85)": float(np.mean(semhits)),
        "BERTScoreF1@1":             None,
    }

    if HAS_BERT_SCORE:
        print("[EVAL] Computing BERTScore ...")
        _, _, F1 = bert_score_fn(pred_as, gold_as, lang="kk", verbose=False)
        results["BERTScoreF1@1"] = float(F1.mean())

    elapsed = time.time() - t0
    print("\n=== Бағалау нәтижелері (TRAIN + EVAL: ТАЗА ДЕРЕКТЕР) ===")
    print(f"  Exact@1:                    {results['Exact@1']:.6f}")
    print(f"  TokenF1@1:                  {results['TokenF1@1']:.6f}")
    print(f"  MeanCos@1(QSim):            {results['MeanCos@1(QSim)']:.6f}")
    print(f"  Semantic@1(ans_cos>=0.85):  {results['Semantic@1(ans_cos>=0.85)']:.6f}")
    if results["BERTScoreF1@1"] is None:
        print("  BERTScoreF1@1:              (pip install bert-score to enable)")
    else:
        print(f"  BERTScoreF1@1:              {results['BERTScoreF1@1']:.6f}")
    print(f"\n  Elapsed: {elapsed:.2f}s | Test samples: {n_test}")
    return results


# ================================================================
# INTERACTIVE DIALOG
# ================================================================
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("Dialog  |  'eval' → бағалау  |  'exit' → шығу")
    print("=" * 60)

    while True:
        user_input = input("\nСұрақ: ").strip()
        if user_input.lower() == "exit":
            print("Stopped. 👋")
            break
        if user_input.lower() == "eval":
            evaluate_system()
            continue
        if not user_input:
            continue
        answer, _, qsim = ask_question(user_input, threshold=0.0)
        print(f"\n  Жауап : {answer}")
        print(f"  QSim  : {qsim:.4f}")